In [1]:
import numpy as np

In [2]:
import pandas as pd

In [4]:
pd.Series([True,False,True,True,False,True,True,False,True,True,False,True]).sum()

8

In [5]:
data = {'Name':['Tom','James','Ricky','Vin','Steve','Minsu','Jack'],
        'Age':[25,26,25,23,30,29,23],
        'Rating':[4.23,3.24,3.98,2.56,3.20,4.6,3.8]
        }

df = pd.DataFrame(data)

In [6]:
df

,Name,Age,Rating
0,Tom,25,4.23
1,James,26,3.24
2,Ricky,25,3.98
3,Vin,23,2.56
4,Steve,30,3.20
5,Minsu,29,4.60
6,Jack,23,3.80


In [7]:
people = pd.Series([100,100],index=[1,3],name='new_Rating')

In [8]:
people

1    100
3    100
Name: new_Rating, dtype: int64

In [9]:
df.Rating * people

0      NaN
1    324.0
2      NaN
3    256.0
4      NaN
5      NaN
6      NaN
dtype: float64

In [10]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
Name      7 non-null object
Age       7 non-null int64
Rating    7 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 621.0 bytes


In [12]:
drinks = pd.read_csv('https://bit.ly/drinksbycountry')

In [13]:
drinks.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa


In [20]:
drinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
country                         193 non-null object
beer_servings                   193 non-null int64
spirit_servings                 193 non-null int64
wine_servings                   193 non-null int64
total_litres_of_pure_alcohol    193 non-null float64
continent                       193 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 9.1+ KB


In [14]:
drinks.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
country                         193 non-null object
beer_servings                   193 non-null int64
spirit_servings                 193 non-null int64
wine_servings                   193 non-null int64
total_litres_of_pure_alcohol    193 non-null float64
continent                       193 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 30.4 KB


In [21]:
drinks.memory_usage()

Index                             80
country                         1544
beer_servings                   1544
spirit_servings                 1544
wine_servings                   1544
total_litres_of_pure_alcohol    1544
continent                       1544
dtype: int64

In [22]:
drinks.memory_usage(deep=True)

Index                              80
country                         12588
beer_servings                    1544
spirit_servings                  1544
wine_servings                    1544
total_litres_of_pure_alcohol     1544
continent                       12332
dtype: int64

In [23]:
drinks.memory_usage(deep=True).sum()

31176

In [17]:
df = drinks.loc[:,['beer_servings','continent']]

In [19]:
df.head()

,beer_servings,continent
0,0,Asia
1,89,Europe
2,25,Africa
3,245,Europe
4,217,Africa


In [24]:
df.memory_usage(deep=True)

Index               80
beer_servings     1544
continent        12332
dtype: int64

In [25]:
df['continent'] = df.continent.astype('category')

In [26]:
df.memory_usage(deep=True)

Index              80
beer_servings    1544
continent         744
dtype: int64

In [27]:
df.continent.head()

0      Asia
1    Europe
2    Africa
3    Europe
4    Africa
Name: continent, dtype: category
Categories (6, object): [Africa, Asia, Europe, North America, Oceania, South America]

In [31]:
df.continent.cat.codes.head()

0    1
1    2
2    0
3    2
4    0
dtype: int8

In [32]:
df.continent.cat.categories

Index(['Africa', 'Asia', 'Europe', 'North America', 'Oceania',
       'South America'],
      dtype='object')